In [12]:
##Imports
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

In [13]:
##Store CSV into DataFrame

csv_file = "../Data/Netflix_titles.csv"
netflix_df = pd.read_csv(csv_file)
netflix_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1060,s1486,Movie,Consequences,Ozan Açıktan,"Nehir Erdoğan, Tardu Flordun, İlker Kaleli, Se...",Turkey,25-Oct-19,2014,TV-MA,106 min,"Dramas, International Movies, Thrillers",Secrets bubble to the surface after a sensual ...
2296,s3283,Movie,Kabhi Khushi Kabhie Gham,Karan Johar,"Amitabh Bachchan, Jaya Bhaduri, Shah Rukh Khan...",India,1-Apr-20,2001,TV-14,209 min,"Dramas, International Movies",Years after his father disowns his adopted bro...
2981,s4280,Movie,Mumbai Meri Jaan,Nishikant Kamat,"Madhavan, Irrfan Khan, Soha Ali Khan, Paresh R...",India,1-Nov-18,2008,TV-MA,135 min,"Dramas, International Movies",A moving portrayal of the 2006 train bombings ...
1083,s1521,Movie,Crazy Awesome Teachers,Sammaria Sari Simanjuntak,"Gading Marten, Dian Sastrowardoyo, Faradina Mu...",Indonesia,17-Aug-20,2020,TV-PG,102 min,"Comedies, Dramas, International Movies","When staff salaries get stolen at his school, ..."
524,s685,Movie,Backtrack,Michael Petroni,"Adrien Brody, Sam Neill, Robin McLeavy, Bruce ...","Australia, United Kingdom, United Arab Emirate...",25-Dec-18,2015,R,90 min,"Dramas, Thrillers","Haunted by visions of the dead, psychologist P..."


In [14]:
##Read Columns

for col in netflix_df.columns: 
    print(col)
    

show_id
type
title
director
cast
country
date_added
release_year
rating
duration
listed_in
description


In [15]:
##Drop all non-Movies and only capture U.S, and movies 2000-2017
netflix_df = netflix_df[(netflix_df.type == "Movie") & (netflix_df.release_year > 1999) & (netflix_df.release_year < 2018)] 
netflix_df.sample(10)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
4119,s5930,Movie,Sustainable,Matt Wechsler,NaN,United States,1-Mar-17,2016,TV-G,92 min,Documentaries,"Chef Rick Bayless, farmers and others discuss ..."
3682,s5297,Movie,Rukh,Atanu Mukherjee,"Adarsh Gourav, Manoj Bajpayee, Smita Tambe, Ku...",India,1-Apr-18,2017,TV-MA,103 min,"Dramas, Independent Movies, International Movies","Following the untimely death of his father, a ..."
1342,s1913,Movie,Easy A,Will Gluck,"Emma Stone, Penn Badgley, Amanda Bynes, Dan By...",United States,1-Nov-20,2010,PG-13,92 min,"Comedies, Romantic Movies",When a lie about Olive's reputation transforms...
5056,s7311,Movie,Umrika,Prashant Nair,"Suraj Sharma, Tony Revolori, Smita Tambe, Adil...",India,1-Oct-16,2015,TV-MA,96 min,"Dramas, Independent Movies, International Movies",An ambitious man leaves his small Indian villa...
904,s1243,Movie,Catch Me If You Can,Steven Spielberg,"Leonardo DiCaprio, Tom Hanks, Christopher Walk...","United States, Canada",1-Jan-21,2002,PG-13,141 min,Dramas,An FBI agent makes it his mission to put cunni...
1300,s1844,Movie,Dr. Seuss' The Lorax,"Chris Renaud, Kyle Balda","Danny DeVito, Ed Helms, Taylor Swift, Zac Efro...","United States, France",7-Jul-20,2012,PG,86 min,Children & Family Movies,The forest-dwelling Lorax has to stop the shor...
25,s34,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,8-Sep-17,2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
2924,s4200,Movie,Monster House,Gil Kenan,"Mitchel Musso, Sam Lerner, Spencer Locke, Stev...",United States,1-Dec-20,2006,PG,91 min,"Children & Family Movies, Comedies",A trio of friends sets out to expose the terro...
5081,s7345,Movie,Unexpectedly Yours,Cathy Garcia-Molina,"Sharon Cuneta, Robin Padilla, Julia Barretto, ...",Philippines,26-Nov-20,2017,TV-14,121 min,"Comedies, International Movies, Romantic Movies",A pair of former batchmates cross paths 30 yea...
1971,s2840,Movie,How the Grinch Stole Christmas,Ron Howard,"Jim Carrey, Taylor Momsen, Jeffrey Tambor, Chr...",United States,1-Jun-17,2000,PG,105 min,"Children & Family Movies, Comedies",The Grinch decides to rob Whoville of Christma...


In [16]:
##Choose Columns, rename 

clean_netflix_df = netflix_df[['title', 'director', 'cast', 'release_year', 'rating', 'duration', 'listed_in', 'description']].copy()
clean_netflix_df = clean_netflix_df.rename(columns={'title': 'movie_title', 'cast': 'actors'})
clean_netflix_df = clean_netflix_df.drop_duplicates(subset='movie_title', keep='first')

clean_netflix_df=clean_netflix_df.drop_duplicates(['movie_title'] , keep ='first') 

clean_netflix_df = clean_netflix_df.reset_index(drop=True)

clean_netflix_df.to_csv("../Output/Netflix_clean.csv")

clean_netflix_df.head(5)

,movie_title,director,actors,release_year,rating,duration,listed_in,description
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
4,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...


In [17]:
#Connect to Local Database

rds_connection_string = "postgres:postgres@localhost:5432/ETL_project_movie_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [18]:
#Check for tables
engine.table_names()

['imbd_data', 'netflix_data']

In [19]:
### Use pandas to load csv converted DataFrame into database
clean_netflix_df.to_sql(name='netflix_data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "netflix_data_pkey"
DETAIL:  Key (movie_title)=(7:19) already exists.

[SQL: INSERT INTO netflix_data (movie_title, director, actors, release_year, rating, duration, listed_in, description) VALUES (%(movie_title)s, %(director)s, %(actors)s, %(release_year)s, %(rating)s, %(duration)s, %(listed_in)s, %(description)s)]
[parameters: ({'movie_title': '7:19', 'director': 'Jorge Michel Grau', 'actors': 'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato', 'release_year': 2016, 'rating': 'TV-MA', 'duration': '93 min', 'listed_in': 'Dramas, International Movies', 'description': 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.'}, {'movie_title': '23:59', 'director': 'Gilbert Chan', 'actors': 'Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai,

In [ ]:
##Confirm data has been added by querying the netflix_data table
pd.read_sql_query('select * from netflix_data', con=engine).head()

In [ ]:
#See how many movies by release year

netflix_by_year = clean_netflix_df["release_year"].value_counts()
netflix_by_year.head()

In [20]:
#See how many movies by Rating

netflix_by_year = clean_netflix_df["rating"].value_counts()
netflix_by_year.head()

TV-MA    997
TV-14    779
R        462
TV-PG    298
PG-13    275
Name: rating, dtype: int64